# Intro to GCP's Gemini 

**Gemini** is a Google multimodal model that has the capability to **summarize, chat, and generate text from images or videos**. Gemini comes in two model versions **Gemini Pro** and **Gemini Pro Vision**, for this tutorial we will be looking into utilizing both models via python packages and GCPs model playground, **Vertex AI Studio**.

## Gemini in Python

The first step inorder to stat using Gemini is to update the google-cloud-aiplatform python package if you havent already.

In [9]:
!pip install --upgrade google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 13.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.37.0
    Uninstalling google-cloud-aiplatform-1.37.0:
      Successfully uninstalled google-cloud-aiplatform-1.37.0


In [59]:
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 5.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.6
    Uninstalling langchain-core-0.1.6:
      Successfully uninstalled langchain-core-0.1.6


Next we initialize the Gemini model by setting out project id and location. We are also pulling in the packages:
- **GenerativeModel:** Allows us to specify and launch the Gemini model we need (e.g. Gemini Pro, Gemini Pro Vision).
- **ChatSession:** Set Gemini Pro in chatbot mode.
- **Part:** Loads in files from buckets.
- **Image:** Loads in image files locally.
- **GenerationConfig:** Allows us to configure the models temperature, top p, top k, and max tokens.

In [6]:
from google.cloud import aiplatform
import vertexai.preview
from vertexai.preview.generative_models import GenerativeModel, ChatSession, Part, Image, GenerationConfig

# TODO(developer): Update and un-comment below lines
project_id = <PROJECT_ID>
location = <LOCATION>
vertexai.init(project=project_id, location=location)

### Gemini as a Chatbot

For dealing with text, code generation, natural language tasks we can use the **gemini-pro** model and to set our model in **chatbot mode** we need to use the `start_chat()` function. You will see below we also created a function named **get_chat_response** which will send the prompt or message we have for our model using the `send_message()` function and returns only the text of the chats response.

In [105]:
model = GenerativeModel("gemini-pro")
chat = model.start_chat()

def get_chat_response(chat: ChatSession, prompt: str):
    response = chat.send_message(prompt)
    return response.text

Now that we have our functions lets ask our Gemini chatbot some questions!

In [ ]:
prompt = "Hello."
print(get_chat_response(chat, prompt))

In [19]:
prompt = "List gen ai use cases that are Life Science or Health Care related. "
print(get_chat_response(chat, prompt))

**Generative AI use cases that are Life Science or Health Care related:**

* **Drug discovery and development:** Generative AI can be used to generate new molecules with desired properties, design new drugs, and predict how drugs will interact with biological systems. This can help to accelerate the drug discovery and development process and make it more efficient.
* **Personalized medicine:** Generative AI can be used to generate personalized treatment plans for patients based on their individual genetic and health data. This can help to improve the effectiveness of treatment and reduce side effects.
* **Disease diagnosis and prognosis:** Generative AI can be used to develop new diagnostic tools and methods, and to predict the course of a disease. This can help to improve patient outcomes and reduce healthcare costs.
* **Medical imaging:** Generative AI can be used to generate synthetic medical images, which can be used to train medical students and residents, develop new imaging tech

We can even ask it to **generate code or debug code**!

In [106]:
prompt = "create a python code that will replace all null values to zero within a csv file"
print(get_chat_response(chat, prompt))

Sure, here's a Python script that will replace all null values (empty cells) with zeros within a CSV file:


```python
import csv

# Open the CSV file for reading and writing.
with open('input.csv', 'r+', newline='') as csvfile:
    # Create a CSV reader and writer.
    reader = csv.reader(csvfile)
    writer = csv.writer(csvfile)

    # Read the header row.
    header = next(reader)

    # Replace null values with zeros in the remaining rows.
    for row in reader:
        for i, cell in enumerate(row):
            if cell == '':
                row[i] = '0'

        # Write the updated row to the CSV file.
        writer.writerow(row)
```


To use this script, save it as a file (e.g. `replace_nulls.py`) and run it from the command line:


```
python replace_nulls.py
```


This will replace all null values in the 'input.csv' file with zeros and create a new CSV file called 'output.csv'.


**Note:** Make sure to replace `input.csv` with the actual name of your input CSV file. You can a

### Gemini as a Summarizer

We can generate text like asking Gemini Pro to summarize articles we provide locally (using langchain). As of now Gemini does not support loading in documents that are not videos and images directly. 

First we will load in a file using langchains text loader. You can also use langchain to load in files from your bucket following the instructions [here](https://python.langchain.com/docs/integrations/document_loaders/google_cloud_storage_file).

In [71]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./PMC10000003.txt")
ex_file=loader.load()

We can configure our model to give us the best optimal output by setting the parameters below:
- **Max_Output_Token**: Max number of words to generate.
- **Temperature:** Controls randomness, higher values increase diversity meaning a more unique response make the model to think harder. Must be a number from 0 to 1.
- **Top_p (nucleus):** The cumulative probability cutoff for token selection. Lower values mean sampling from a smaller, more top-weighted nucleus. Must be a number from 0 to 1.
- **Top_k:** Sample from the k most likely next tokens at each step. Lower k focuses on higher probability tokens. This means the model choses the most probable words. Lower values eliminate fewer coherent words.


In [103]:
generation_config = GenerationConfig(
    temperature=0.9,
    top_p=1.0,
    top_k=32,
    candidate_count=1,
    max_output_tokens=8192,
)

def summarizer(file: str) -> str:
        
    # Query the model
    response = model.generate_content(
        [
            # Add an example query
            "summarize this file.",
            file
        ],
        generation_config=generation_config,
    )
    #print(response)
    return response.text

Here we are inputting only the page content from our document loader.

In [92]:
print(summarizer(ex_file[0].page_content))

The article "Mechanical Treatment of Inflammation of the Knee Joint" published in the Chicago Medical Examiner on January 1867, discusses the use of mechanical treatments for inflammation of the knee joint. The author emphasizes the importance of overcoming the reflex contraction of muscles surrounding the joint to prevent or correct deformities. Tenotomy of the flexor tendons may be necessary to achieve this. Additionally, the relief of pressure on the inflamed joint surfaces is crucial for recovery. This can be achieved through various methods such as adhesive strap dressings, application of an air cushion, or evacuation of pus from the joint. The author also introduces a new apparatus for making extension, which allows for optimal counter-extension and can be used in both acute and chronic cases. The advantages of this apparatus include its large counter-extending surface, security, and patient comfort. By utilizing this principle, various instruments can be crafted to address knee 

### Gemini as a Image to Text Generator

Gemini Pro Vision can generate text from images and videos. These text can be descriptions or questions about the image or video. You can download an image or retrieve an image from your bucket or locally.

Images can only be in the following formats: 
- PNG - image/png
- JPEG - image/jpeg

Our function below takes in a prompt and the image, we have also included a if statement to recognize if the function should use `Image` to load in a image locally or `Part` to load it from a bucket.

In [ ]:
def img2text(image_path: str, img_prompt: str) -> str:
    multimodal_model = GenerativeModel("gemini-pro-vision")
    if "gs://" in image_path:
        image1=Part.from_uri(image_path, mime_type="image/jpeg")
    else: 
        image1=Image.load_from_file(image_path)
    #image1=Image.load_from_file(image_path)
    responses = multimodal_model.generate_content(
        [image1, img_prompt],
        generation_config={
            "max_output_tokens": 2048,
            "temperature": 0.4,
            "top_p": 1,
            "top_k": 32
        },
        stream=True,
    )
    for response in responses:
        print(response.text, end="")

Lets look at an image locally, by loading a image first, this a image of a Covid virus from the [CDC Public Health Image Library](https://phil.cdc.gov/details.aspx?pid=23312).

In [146]:
! wget -O example_image_covid.jpg "https://phil.cdc.gov//PHIL_Images/23312/23312_lores.jpg" 

--2024-01-11 05:24:55--  https://phil.cdc.gov//PHIL_Images/23312/23312_lores.jpg
Resolving phil.cdc.gov (phil.cdc.gov)... 198.246.102.26
Connecting to phil.cdc.gov (phil.cdc.gov)|198.246.102.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31823 (31K) [image/jpeg]
Saving to: ‘example_image_covid.jpg’

example_image_covid 100%[===================>]  31.08K  --.-KB/s    in 0.07s   

2024-01-11 05:24:55 (455 KB/s) - ‘example_image_covid.jpg’ saved [31823/31823]



Now run our function!

In [180]:
print(img2text("example_image_covid.jpg", "describe this image."))

 The image is a 3D rendering of a coronavirus. The virus is round and has a spiky outer coat. The spikes are made of proteins that help the virus attach to and infect cells. The virus is colored gray and red.None


Next we'll look at an image from a bucket.

In [181]:
print(img2text("gs://generativeai-downloads/images/scones.jpg", "describe this image."))

 The image shows a table with a pink peony bouquet, two cups of coffee, a bowl of blueberries, and a silver spoon with the words "Let's Jam" on it. There are also five scones with blueberries on them. The table is covered with a white tablecloth with purple stains.None


We can even ask for more details related to the items in our image!

In [182]:
img_prompt="How do you make whats in this image?"
image="gs://generativeai-downloads/images/scones.jpg"
print(img2text(image, img_prompt))

 Preheat oven to 375 degrees F (190 degrees C). Grease a baking sheet.

In a large bowl, combine the flour, sugar, baking powder, and salt. Cut in butter until mixture resembles coarse crumbs. Stir in blueberries.

Turn out onto a lightly floured surface; knead for 10 to 12 times. Pat into a 1/2-in.-thick circle. Cut with a 3-in. floured biscuit cutter. Place 2 in. apart on the prepared baking sheet.

Bake for 12-15 minutes or until golden brown. Cool for 2 minutes before removing to a wire rack to cool completely.None


### Gemini as a Video to Text Generator

Just like images we will be using the same model Gemini Pro Vision. We can load videos locally and from a bucket just like images. Video files can only be in the following formats:
- MOV - video/mov
- MPEG - video/mpeg
- MP4 - video/mp4
- MPG - video/mpg
- AVI - video/avi
- WMV - video/wmv
- MPEGPS - video/mpegps
- FLS - video/flv

Our function below takes a video from a public bucket and asks for a prompt and the location of the video file.

In [ ]:
def video2text(video_path: str, video_prompt: str) -> str:
    # Query the model
    response = multimodal_model.generate_content(
        [
            # Add an example image
            Part.from_uri(
                video_path, mime_type="video/mp4"
            ),
            # Add an example query
            video_prompt,
        ],
        stream=True
    )
    for chunk in response :
        return print(chunk.text)


Run the function!

In [191]:
video_prompt="What is this video about in detail?"
video="gs://cloud-samples-data/video/Machine Learning Solving Problems Big, Small, and Prickly.mp4"
print(video2text(video, video_prompt))

 This video is about a messy world. It shows a bunch of different things that are messy, like a messy room, a messy desk, and a messy
None


## Gemini on Vertex AI Studio

You can also use Gemini Pro and Pro Vision in Vertex AI's playground called **Vertex AI Studio**. To locate Vertex AI Studio search Vertex AI and on the left hand side locate Vertex AI Studio as the image below shows. To utilize Gemini Pro Vision locate and click **Multimodal** you will have the option to use your own prompt or explore some of the other set prompts such as Extract text from images, image question answering , etc.

![Gemini1](../../../images/Gemini_1.png)

For this tutorial we will select Open on the **Prompt Design** option. We will upload the COVID image we downloaded before by clicking **INSERT MEDIA** and selecting our file. Then we will ask it a question, here we asked "Describe treatments for the item in this image".

![Gemini3](../../../images/Gemini_3.png)

To utilize Gemini Pro locate and click **Language** on the left side menu. You have the option to use a prompt or chat and if you would like to focus on text or code.

![Gemini2](../../../images/Gemini_2.png)

Here we picked the **TEXT CHAT** option and asked the bot to describe covid and how it works.

![Gemini4](../../../images/Gemini_4.png)